In [1]:
import pandas as pd

# Load the data from the JSON file
complaints_data = pd.read_json("complaints-2023-08-25_18_02.json")

# Display the first few rows of the data
complaints_data.head()

,_index,_type,_id,_score,_source,sort
0,complaint-public-v2,_doc,5471601,NaN,"{'product': 'Checking or savings account', 'co...",[16]
1,complaint-public-v2,_doc,5020019,NaN,"{'product': 'Checking or savings account', 'co...",[108]
2,complaint-public-v2,_doc,7203230,NaN,"{'product': 'Vehicle loan or lease', 'complain...",[136]
3,complaint-public-v2,_doc,3743284,NaN,"{'product': 'Checking or savings account', 'co...",[156]
4,complaint-public-v2,_doc,2927362,NaN,"{'product': 'Credit reporting, credit repair s...",[188]


In [3]:
# Extract a sample record from the _source column
sample_source_record = complaints_data["_source"].iloc[1]

sample_source_record

{'product': 'Checking or savings account',
 'complaint_what_happened': 'I opened two accounts with Ally Bank on XX/XX/XXXX of 2021, depositing {$5000.00} into an interest checking account and {$5000.00} into a savings account. I received a debit card in the mail. \n\nAfter a few weeks I tried my first transaction, which I think was a small debit card purchase. My account was immediately locked for suspected fraud... I received notification in the mail. \n\nI phone Ally Bank on XX/XX/XXXX and aske for the fraud department, where I encountered " XXXX, \'\' a XXXX XXXX XXXX XXXX XXXX  employee who would tell me nothing and only said he would send a document upload link via email, then hung up on me. \n\nI uploaded all the requested documents, then waited several weeks. Nothing happened. \n\nI called back and spoke with XXXX in fraud prevention. She was apologetic, but said none of my uploaded documents were in the file. She didn\'t know why. She sent me another upload load and promised to

Distribution of Products

In [4]:
# Convert the data into a pandas DataFrame
df = pd.DataFrame(complaints_data)

# Extract the 'product' column from the '_source' dictionary column
df['product'] = df['_source'].apply(lambda x: x['product'])

# Count the occurrences of each product
product_counts = df['product'].value_counts(dropna=False)

product_counts.name = 'Counts'

product_counts

Credit reporting, credit repair services, or other personal consumer reports    4214
Vehicle loan or lease                                                           3542
Checking or savings account                                                     2283
Consumer Loan                                                                   2094
Debt collection                                                                 1187
Bank account or service                                                          777
Mortgage                                                                         295
Credit card or prepaid card                                                      216
Money transfer, virtual currency, or money service                               189
Payday loan, title loan, or personal loan                                        154
Credit reporting                                                                  37
Credit card                                                      

In [5]:
# Count the frequrncy rate of each product
product_percents = df['product'].value_counts(normalize=True).map(lambda x: f'{x*100:.2f}%')
product_percents.name = 'Percents'
product_percents

Credit reporting, credit repair services, or other personal consumer reports    28.05%
Vehicle loan or lease                                                           23.58%
Checking or savings account                                                     15.20%
Consumer Loan                                                                   13.94%
Debt collection                                                                  7.90%
Bank account or service                                                          5.17%
Mortgage                                                                         1.96%
Credit card or prepaid card                                                      1.44%
Money transfer, virtual currency, or money service                               1.26%
Payday loan, title loan, or personal loan                                        1.03%
Credit reporting                                                                 0.25%
Credit card                                

In [6]:
# Combine the Series into a DataFrame
df_products = pd.concat([product_counts, product_percents], axis=1)
df_products

,Counts,Percents
"Credit reporting, credit repair services, or other personal consumer reports",4214,28.05%
Vehicle loan or lease,3542,23.58%
Checking or savings account,2283,15.20%
Consumer Loan,2094,13.94%
Debt collection,1187,7.90%
Bank account or service,777,5.17%
Mortgage,295,1.96%
Credit card or prepaid card,216,1.44%
"Money transfer, virtual currency, or money service",189,1.26%
"Payday loan, title loan, or personal loan",154,1.03%


In [7]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add bar trace for Counts
fig.add_trace(go.Bar(x=df_products.index, y=df_products['Counts'], name='Counts'))

# Add line trace for Percents, set to secondary y-axis
# Convert percentages to float for plotting
df_products['Percents'] = df_products['Percents'].str.rstrip('%').astype('float') / 100.0
fig.add_trace(go.Scatter(x=df_products.index, y=df_products['Percents'], name='Percents', yaxis='y2', mode='lines+markers'))

# Set up the layout, including dual y-axes
fig.update_layout(
    yaxis=dict(title='Counts'),
    yaxis2=dict(title='Percents', overlaying='y', side='right'),
    title="Distribution of Products",
    hovermode='x unified',  # Show hover for all traces at the same x value
    autosize=True,
    margin=dict(autoexpand=True),  # Adjust margins to make the graph size adjustable
    width=600,  # Initial width
    height=400   # Initial height
)

# Hide x-axis tick labels
fig.update_xaxes(showticklabels=False)

fig.show()

Distribution of Issues by Products

In [8]:
# Extract the 'product' and 'issue' columns from the '_source' dictionary column
df['product'] = df['_source'].apply(lambda x: x.get('product', None))
df['issue'] = df['_source'].apply(lambda x: x.get('issue', None))

In [9]:
# Group by the 'product' and 'issue' columns and count the occurrences of each combination
issue_by_product = df.groupby(['product', 'issue']).size().reset_index(name='count')
issue_by_product

,product,issue,count
0,Bank account or service,"Account opening, closing, or management",427
1,Bank account or service,Deposits and withdrawals,189
2,Bank account or service,"Making/receiving payments, sending money",82
3,Bank account or service,Problems caused by my funds being low,42
4,Bank account or service,Using a debit or ATM card,37
...,...,...,...
131,Vehicle loan or lease,Problem with a credit reporting company's inve...,133
132,Vehicle loan or lease,Problem with fraud alerts or security freezes,1
133,Vehicle loan or lease,Problems at the end of the loan or lease,823
134,Vehicle loan or lease,Struggling to pay your loan,558


In [10]:
#Extract the Top 7 Products:
top_7_products = product_counts.nlargest(7)
top_7_products

Credit reporting, credit repair services, or other personal consumer reports    4214
Vehicle loan or lease                                                           3542
Checking or savings account                                                     2283
Consumer Loan                                                                   2094
Debt collection                                                                 1187
Bank account or service                                                          777
Mortgage                                                                         295
Name: Counts, dtype: int64

In [11]:
#Filter the issue_by_product DataFrame for Only These Products
filtered_issues = issue_by_product[issue_by_product['product'].isin(top_7_products.index)]
filtered_issues

,product,issue,count
0,Bank account or service,"Account opening, closing, or management",427
1,Bank account or service,Deposits and withdrawals,189
2,Bank account or service,"Making/receiving payments, sending money",82
3,Bank account or service,Problems caused by my funds being low,42
4,Bank account or service,Using a debit or ATM card,37
...,...,...,...
131,Vehicle loan or lease,Problem with a credit reporting company's inve...,133
132,Vehicle loan or lease,Problem with fraud alerts or security freezes,1
133,Vehicle loan or lease,Problems at the end of the loan or lease,823
134,Vehicle loan or lease,Struggling to pay your loan,558


In [12]:
#Calculate the Rate of Each Issue as a Percentage of Each Product's Number
merged_df = filtered_issues.merge(top_7_products.rename('product_total'), left_on='product', right_index=True)
merged_df['issue_rate'] = (merged_df['count'] / merged_df['product_total']) * 100
merged_df

,product,issue,count,product_total,issue_rate
0,Bank account or service,"Account opening, closing, or management",427,777,54.954955
1,Bank account or service,Deposits and withdrawals,189,777,24.324324
2,Bank account or service,"Making/receiving payments, sending money",82,777,10.553411
3,Bank account or service,Problems caused by my funds being low,42,777,5.405405
4,Bank account or service,Using a debit or ATM card,37,777,4.761905
...,...,...,...,...,...
131,Vehicle loan or lease,Problem with a credit reporting company's inve...,133,3542,3.754941
132,Vehicle loan or lease,Problem with fraud alerts or security freezes,1,3542,0.028233
133,Vehicle loan or lease,Problems at the end of the loan or lease,823,3542,23.235460
134,Vehicle loan or lease,Struggling to pay your loan,558,3542,15.753811


In [13]:
# Sort merged_df by product_total and then issue_rate
sorted_merged_df = merged_df.sort_values(by=['product_total', 'issue_rate'], ascending=[False, False])
sorted_merged_df

,product,issue,count,product_total,issue_rate
53,"Credit reporting, credit repair services, or o...",Improper use of your report,1819,4214,43.165638
54,"Credit reporting, credit repair services, or o...",Incorrect information on your report,1573,4214,37.327954
56,"Credit reporting, credit repair services, or o...",Problem with a credit reporting company's inve...,759,4214,18.011391
49,"Credit reporting, credit repair services, or o...",Credit monitoring or identity theft protection...,29,4214,0.688182
59,"Credit reporting, credit repair services, or o...",Unable to get your credit report or credit score,8,4214,0.189843
...,...,...,...,...,...
93,Mortgage,Credit decision / Underwriting,3,295,1.016949
98,Mortgage,Other,2,295,0.677966
99,Mortgage,Problem with a credit reporting company's inve...,2,295,0.677966
94,Mortgage,Improper use of your report,1,295,0.338983


In [14]:
# Sort by product_total and then issue_rate
sorted_merged_df = merged_df.sort_values(by=['product_total', 'issue_rate'], ascending=[False, False])

# Compute the cumulative sum of issue_rate within each product
sorted_merged_df['cumulative_issue_rate'] = sorted_merged_df.groupby('product')['issue_rate'].cumsum()

# Filter rows where cumulative sum is less than or equal to 90%
final_df = sorted_merged_df[sorted_merged_df['cumulative_issue_rate'] <= 90]

final_df

,product,issue,count,product_total,issue_rate,cumulative_issue_rate
53,"Credit reporting, credit repair services, or o...",Improper use of your report,1819,4214,43.165638,43.165638
54,"Credit reporting, credit repair services, or o...",Incorrect information on your report,1573,4214,37.327954,80.493593
130,Vehicle loan or lease,Managing the loan or lease,1211,3542,34.189723,34.189723
133,Vehicle loan or lease,Problems at the end of the loan or lease,823,3542,23.235460,57.425184
134,Vehicle loan or lease,Struggling to pay your loan,558,3542,15.753811,73.178995
127,Vehicle loan or lease,Getting a loan or lease,413,3542,11.660079,84.839074
8,Checking or savings account,Managing an account,1626,2283,71.222076,71.222076
5,Checking or savings account,Closing an account,286,2283,12.527376,83.749452
14,Consumer Loan,Managing the loan or lease,1291,2094,61.652340,61.652340
16,Consumer Loan,Problems when you are unable to pay,423,2094,20.200573,81.852913


In [28]:
import plotly.express as px

# Combine issue and product columns to form a new categorical x-axis
final_df['combined'] = final_df['product'] + " - " + final_df['issue']

# Sort the dataframe by product_totals and issue_rate to maintain order in the chart
final_df = final_df.sort_values(by=['product_total', 'issue_rate'], ascending=[False, True])

# Colors
colors = px.colors.qualitative.Set1

# Mapping products to colors
product_to_color = {product: colors[idx] for idx, product in enumerate(final_df['product'].unique())}

fig = go.Figure()

# Plotted both bars and lines in a single loop for each product
for product in final_df['product'].unique():
    subset_df = final_df[final_df['product'] == product]
    
    # Bar for counts
    fig.add_trace(go.Bar(
        x=subset_df['combined'],
        y=subset_df['count'],
        name=product,
        marker_color=product_to_color[product],
        hoverinfo='y+text',
        text=subset_df['combined']
    ))
    
    # Line for issue rate
    fig.add_trace(go.Scatter(
        x=subset_df['combined'],
        y=subset_df['issue_rate'],
        yaxis='y2',
        mode='lines+markers',
        line=dict(color=product_to_color[product]),
        name=product,
        hoverinfo='y+text',
        text=subset_df['combined'],
        showlegend=False
    ))

# Updating layout
fig.update_layout(
    title="Issue Counts and Issue Rates by Product",
    xaxis_title="Products and Issues",
    yaxis_title="Count",
    yaxis2=dict(title="Issue Rate (%)", overlaying='y', side='right'),
    hovermode="x unified",
    xaxis=dict(showticklabels=False),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()


Distribution of Complaints by states

In [2]:
# Importing the required libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Extracting the 'state' information from the '_source' column
complaints_data['state'] = complaints_data['_source'].apply(lambda x: x.get('state', None))

# Checking for missing values in 'state' column
missing_states = complaints_data['state'].isna().sum()

missing_states


168

In [ ]:
complaints-2023-08-25_18_02.json
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

In [11]:
import pandas as pd

# Load the complaints data (replace this with the path to your JSON file)
complaints_data = pd.read_json("complaints-2023-08-25_18_02.json")

# Extract the 'state' information from the '_source' column
complaints_data['state'] = complaints_data['_source'].apply(lambda x: x.get('state', None))

# Aggregate the complaints data by state and count
complaints_by_state = complaints_data['state'].value_counts().reset_index()
complaints_by_state.columns = ['state', 'count']

# Create a list of all US states (abbreviations)
usa_states = [
    'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 
    'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 
    'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 
    'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 
    'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'
]

# Add rows for missing states with a count of 0
missing_states = set(usa_states) - set(complaints_by_state['state'])
for state in missing_states:
    complaints_by_state = complaints_by_state.append({'state': state, 'count': 0}, ignore_index=True)

# Sort by state for easier inspection
complaints_by_state = complaints_by_state.sort_values('state').reset_index(drop=True)

print(complaints_by_state)


   state  count
0     AE      3
1     AK     15
2     AL    194
3     AR    138
4     AZ    351
5     CA   1551
6     CO    206
7     CT    193
8     DC     71
9     DE    104
10    FL   1529
11    FM      1
12    GA    867
13    GU      1
14    HI     30
15    IA     49
16    ID     28
17    IL    672
18    IN    155
19    KS     67
20    KY     85
21    LA    185
22    MA    318
23    MD    423
24    ME     38
25    MI    554
26    MN    137
27    MO    221
28    MS    133
29    MT     23
30    NC    517
31    ND     24
32    NE     33
33    NH     32
34    NJ    606
35    NM     43
36    NV    192
37    NY    822
38    OH    553
39    OK    117
40    OR    105
41    PA    712
42    PR      5
43    RI     53
44    SC    226
45    SD     19
46    TN    298
47    TX   1332
48    UT     59
49    VA    364
50    VT     18
51    WA    176
52    WI    136
53    WV     51
54    WY     18


In [14]:
import json

# Load GeoJSON data for U.S. states (replace this with the path to your GeoJSON file)
us_states = json.load(open("gz_2010_us_040_00_500k.json"))

# Print properties of the first feature
print(us_states['features'][0]['properties'])


{'GEO_ID': '0400000US23', 'STATE': '23', 'NAME': 'Maine', 'LSAD': '', 'CENSUSAREA': 30842.923}


In [19]:
import plotly.express as px

# Create the choropleth map
fig = px.choropleth(complaints_by_state, 
                    locations='state', 
                    color='count',
                    locationmode='USA-states',
                    scope='usa',
                    color_continuous_scale='Blues',
                    labels={'count':'Number of Complaints'},
                    hover_name='state',
                    hover_data=['count'],
                    title='Distribution of Complaints by State in the USA')

# Show the map
fig.show()